# ILLIXR Log to CSV

In [2]:
import re
import pandas as pd

def timewarp_gl_to_csv(log_filepath):
    # Regular expression pattern for timewarp_gl entries
    pattern_timewarp = r'\[(.*?)\] \[timewarp_gl\] \[debug\] <RTEN> timewarp_gl (.*?): (\d+\.\d+)'

    # Dictionary to keep track of start times
    timewarp_starts = {}

    # List for parsed data
    parsed_data_timewarp = []

    # Read the log file
    with open(log_filepath, 'r') as file:
        for line in file:
            match = re.search(pattern_timewarp, line)
            if match:
                timestamp, phase, rt = match.groups()
                if phase.endswith('begin'):
                    timewarp_starts[timestamp] = rt
                elif phase.endswith('end'):
                    # Find the corresponding start timestamp
                    start_ts = next(iter(timewarp_starts))
                    parsed_data_timewarp.append({
                        'Task': 'timewarp_gl',
                        'start_rt': timewarp_starts.pop(start_ts),
                        'finish_rt': rt,
                        'start_ts': start_ts,
                        'finish_ts': timestamp
                    })

    # Create DataFrame and save to CSV for timewarp_gl
    df_timewarp = pd.DataFrame(parsed_data_timewarp)
    df_timewarp.to_csv('timewarp_output.csv', index=False)

def gldemo_to_csv(log_filepath):
    # Regular expression pattern for gldemo entries
    pattern_gldemo = r'\[(.*?)\] \[gldemo\] \[debug\] <RTEN> gldemo (.*?): (\d+\.\d+)'

    # Dictionary to keep track of 'begin after vsync' start times
    gldemo_starts = {}

    # List for parsed data
    parsed_data_gldemo = []

    # Read the log file
    with open(log_filepath, 'r') as file:
        for line in file:
            match = re.search(pattern_gldemo, line)
            if match:
                timestamp, phase, rt = match.groups()
                if phase == 'begin after vsync':
                    gldemo_starts[timestamp] = rt
                elif phase == 'begin before vsync' and gldemo_starts:
                    # Find the corresponding start timestamp
                    start_ts = next(iter(gldemo_starts))
                    parsed_data_gldemo.append({
                        'Task': 'gldemo',
                        'start_rt': gldemo_starts.pop(start_ts),
                        'finish_rt': rt,
                        'start_ts': start_ts,
                        'finish_ts': timestamp
                    })

    # Create DataFrame and save to CSV for gldemo
    df_gldemo = pd.DataFrame(parsed_data_gldemo)
    df_gldemo.to_csv('gldemo_output.csv', index=False)


In [3]:
def parse_ILLIXR_to_csv(log_filepath):
    gldemo_to_csv(log_filepath)
    timewarp_gl_to_csv(log_filepath)

In [4]:
if __name__ == '__main__':
    parse_ILLIXR_to_csv('./MH01_20240129_8C30W/log_MH01_2.txt')